In [98]:
import io
import re
from datetime import datetime

import dr_util.file_utils as fu
import roam_man.process_utils as pu
from roam_man.roam_graph import RoamGraph, roam_data_to_full_str

In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Util Definitions

In [62]:
def is_page_empty(page):
    if len(page.children) == 0:
        return True

    # Drop the title
    page_strs = page.to_str_full(verbose=False).split("\n")[1:]
    
    for ps in page_strs:
        psreal = ps.strip().strip("-")
        if len(psreal) > 0:
            return False

    return True

In [83]:
# Helper function to parse date strings like "May 8th, 2024"
def parse_date(date_str):
    # Remove the ordinal suffix from the day part (e.g., "8th" -> "8")
    cleaned_date_str = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str)
    # Parse the cleaned date string into a datetime object
    return datetime.strptime(cleaned_date_str, "%B %d, %Y")

In [93]:
def get_pages_by_year(graph, year='all', drop_empty=False):
    title_sets = pu.page_node_list_to_title_sets(graph.roam_pages, graph.uid_to_title)
    dp = title_sets['daily_pages']
    if year == 'all':
        this_year = dp
    else:
        this_year = [pt for pt in dp if year in pt]
    ty_sorted = sorted(this_year, key=lambda st: parse_date(st))
    nodes = [graph.get_page_node(pt) for pt in ty_sorted]
    if drop_empty:
        nodes = [n for n in nodes if not is_page_empty(n)]
    return nodes

In [95]:
def merge_all_daily_pages(graph, year='all'):
    tydp_nonempty = get_pages_by_year(graph, year=year, drop_empty=True)
    buffer = io.StringIO()
    for dp in tydp_nonempty:
        buffer.write(dp.to_str_full(verbose=False))
        buffer.write("\n\n")
    rep = buffer.getvalue()
    buffer.close()
    return rep

### Testing the Utils

## The Ultimate Goal: Import graph dump formatted non-empty daily pages

In [3]:
rg = RoamGraph("/Users/daniellerothermel/Desktop/drotherm_roam-2024-10-02-15-16-48.json")

In [4]:
title_sets = pu.page_node_list_to_title_sets(rg.roam_pages, rg.uid_to_title)

In [101]:
# The Ultimate Goal: And it Worked!!!
all_dps = merge_all_daily_pages(rg, year='2024')
fu.dump_file(all_dps, '/Users/daniellerothermel/Desktop/all_daily_pages_2024_drotherm_roam_2024.10.02.txt')

True